In [3]:
import sys
import os
import time  # 렌더링 간 딜레이를 추가하기 위해 사용
from environments.environment import RushHourEnv
from trains.train import train_dqn
from utils.visualize import plot_rewards_and_steps
from environments.vehicle_data import vehicle_data  # vehicle_data 가져오기

# 프로젝트 루트를 Python 경로에 추가
project_root = os.path.abspath(".")
sys.path.append(project_root)

# vehicle_data를 인자로 전달하여 환경 생성
env = RushHourEnv(vehicle_data)

# 학습 수행
rewards, steps, last_episode_steps, max_reward, max_reward_episode = train_dqn(env, num_episodes=100)  # 마지막 에피소드 데이터 반환
print("Last Episode Steps: ", last_episode_steps)
print(f"Maximum Reward: {max_reward} at Episode {max_reward_episode}")

# 결과 시각화
plot_rewards_and_steps(rewards, steps)
'''
# 마지막 에피소드 렌더링
env.reset()  # 환경 초기화
for state, action in last_episode_steps:
    env.render()  # 현재 상태 렌더링
    time.sleep(0.05)  # 렌더링 간 딜레이
    env.step(action)  # 저장된 행동 수행
env.render()  # 마지막 상태 렌더링
'''



Cannot move vehicle at (6, 3) due to obstacles or invalid move.
Cannot move vehicle at (1, 4) due to obstacles or invalid move.
Cannot move vehicle at (1, 3) due to obstacles or invalid move.
Vehicle at (4, 1) moved 0 step(s).
Vehicle at (4, 1) moved 0 step(s).
Cannot move vehicle at (2, 6) due to obstacles or invalid move.
Cannot move vehicle at (1, 3) due to obstacles or invalid move.
Cannot move vehicle at (4, 6) due to obstacles or invalid move.
Cannot move vehicle at (4, 6) due to obstacles or invalid move.
Cannot move vehicle at (6, 3) due to obstacles or invalid move.
Vehicle at (6, 2) moved 0 step(s).
Cannot move vehicle at (4, 6) due to obstacles or invalid move.
Cannot move vehicle at (2, 6) due to obstacles or invalid move.
Cannot move vehicle at (1, 4) due to obstacles or invalid move.
Cannot move vehicle at (2, 6) due to obstacles or invalid move.
Cannot move vehicle at (1, 3) due to obstacles or invalid move.
Cannot move vehicle at (1, 4) due to obstacles or invalid move.

In [ ]:
import time
import numpy as np

# 상태 비교 함수 - 이동 여부 확인
def has_state_changed(previous_state, current_state):
    """이전 상태와 현재 상태를 비교하여 변화 여부를 확인"""
    return not np.array_equal(previous_state, current_state)

# 렌더링 함수
def render_steps(env, step_data):
    """저장된 상태-액션 쌍을 기반으로 렌더링"""
    previous_state = env.get_observation()  # 초기 상태 저장
    
    for i, (state, action) in enumerate(step_data):
        print(f"Rendering Step {i + 1} with Action: {action}")
        
        # 상태 비교 (이전 상태와 현재 상태가 다를 때만 렌더링)
        if has_state_changed(previous_state, state):
            env.render()  # 상태가 변화한 경우에만 렌더링
            previous_state = state  # 현재 상태를 이전 상태로 갱신
            env.step(action)  # 저장된 행동을 환경에 적용
            time.sleep(0.5)  # 렌더링 간 딜레이

# 학습된 데이터를 바탕으로 마지막 에피소드 렌더링
env.reset()  # 환경 초기화
render_steps(env, last_episode_steps)
# env.render()  # 마지막 상태 렌더링


NameError: name 'env' is not defined

In [ ]:
현재 문제 실패하는 행위가 너무 많이 보이고 있다.
